In [14]:
import pandas as pd
import requests
import time
import folium
from folium.plugins import MarkerCluster


#### This code takes up to 20 mins (API)

In [1]:
##### DONT RUN THIS IF YOU ARE IN HURRY!!#######
##### This code only needs to be run once for long and lat data from the slaughterhouses #########################
# Step 1: Read the CSV file
df = pd.read_csv('slaughterhouse.csv', sep=';')

# OpenCage API key
api_key = '9dfdee8dfdc24fe0b8df509e9cd055d6'  # appi key

# Function to get coordinates using OpenCage Geocoding API
def get_coordinates(address):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['geometry']['lat'], data['results'][0]['geometry']['lng']
    return None, None

# Geocode addresses
coordinates = []
for index, row in df.iterrows():
    address = f"{row['Adresse']}, {row['PLZ']} {row['Ort/Region']}, Switzerland"
    latitude, longitude = get_coordinates(address)
    if latitude and longitude:
        coordinates.append((latitude, longitude))
    else:
        coordinates.append((None, None))
    time.sleep(1)  # Add delay to respect usage limits

df['Latitude'], df['Longitude'] = zip(*coordinates)

# Save the coordinates to a new CSV file
df.to_csv('slaughterhouse_with_coordinates.csv', index=False)



NameError: name 'pd' is not defined

## Overview of the Abbreviation ####

In [10]:
# German
animal_mapping_german = {
    'A': 'Hausgeflügel',
    'B': 'Rindergattung',
    'C': 'Ziegenartige',
    'L': 'Hasenartige (domestiziert)',
    'O': 'Schafe',
    'P': 'Schweine',
    'S': 'Einhufer',
    'fG': 'Im Gehege gehaltene Landsäuger',
    'R': 'Laufvögel',
    'wA': 'Wildvögel',
    'wL': 'Wilde Hasenartige',
    'wU': 'Wilde Huftiere',
    'wG': 'Wilde Landsäuger'
}


# French
animal_mapping_french = {
    'A': 'Volaille domestique',
    'B': 'Bovins',
    'C': 'Caprins',
    'L': 'Léporidés (domestiques)',
    'O': 'Moutons',
    'P': 'Porcs',
    'S': 'Équidés',
    'fG': 'Mammifères terrestres en enclos',
    'R': 'Oiseaux coureurs',
    'wA': 'Oiseaux sauvages',
    'wL': 'Léporidés sauvages',
    'wU': 'Ongulés sauvages',
    'wG': 'Mammifères terrestres sauvages'
}



#### German version (produces germam html)

In [30]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

# Define the animal mappings in German
animal_mapping_german = {
    'A': 'Hausgeflügel',
    'B': 'Rindergattung',
    'C': 'Ziegenartige',
    'L': 'Hasenartige (domestiziert)',
    'O': 'Schafe',
    'P': 'Schweine',
    'S': 'Einhufer',
    'fG': 'Im Gehege gehaltene Landsäuger',
    'R': 'Laufvögel',
    'wA': 'Wildvögel',
    'wL': 'Wilde Hasenartige',
    'wU': 'Wilde Huftiere',
    'wG': 'Wilde Landsäuger'
}

# Correct file paths for the provided icons
cow_icon_path = 'cowhead.png'
goat_icon_path = 'goathead.png'
sheep_icon_path = 'sheephead.png'

# Load the CSV file
file_path = 'slaughterhouse_with_coordinates.csv'
slaughterhouse_data = pd.read_csv(file_path)

# Clean the data: remove rows where 'Tierart' is NaN or not a string, and where Latitude or Longitude are NaN
slaughterhouse_data_cleaned = slaughterhouse_data.dropna(subset=['Tierart', 'Latitude', 'Longitude'])
slaughterhouse_data_cleaned = slaughterhouse_data_cleaned[slaughterhouse_data_cleaned['Tierart'].apply(lambda x: isinstance(x, str))]

# Define the bounding box for Switzerland
switzerland_bounds = [[45.8179, 5.9561], [47.8085, 10.4923]]

# Create a function to generate the map based on the animal type and icon
def create_animal_map(animal_type, icon_path, animal_name):
    # Filter data for the specific animal type
    filtered_data = slaughterhouse_data_cleaned[slaughterhouse_data_cleaned['Tierart'].str.contains(animal_type)]

    # Create a base map centered around Switzerland with max bounds
    map_center = [46.8182, 8.2275]  # Latitude and Longitude of Switzerland
    m = folium.Map(
        location=map_center, 
        zoom_start=7.5,
        min_zoom=7.5,
        max_bounds=True,
        max_lat=47.8085, min_lat=45.8179,
        max_lon=10.4923, min_lon=5.9561
    )

    # Add max bounds to the map
    m.fit_bounds(switzerland_bounds)
    m.add_child(folium.LatLngPopup())  # Optional: for debugging to see lat/lon

    # Create a marker cluster
    marker_cluster = MarkerCluster().add_to(m)

    # Add markers to the map
    for index, row in filtered_data.iterrows():
        popup_content = (
            f"<b>{row['Firmenname']}</b><br>{row['Adresse']}<br>{int(row['PLZ'])} {row['Ort/Region']}<br>"
            f"Services: {', '.join([animal_mapping_german[type] for type in row['Tierart'].split(', ') if type in animal_mapping_german])}"
        )
        popup = folium.Popup(popup_content, max_width=300)

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup,
            icon=folium.CustomIcon(icon_path, icon_size=(30, 30))
        ).add_to(marker_cluster)

    # Save the map to an HTML file
    output_file_path = f'slaughterhouses_map_{animal_name}_de.html'
    m.save(output_file_path)

    return output_file_path

# Create the maps for cattle, goats, and sheep
cattle_map_path = create_animal_map('B', cow_icon_path, 'cattle')
goat_map_path = create_animal_map('C', goat_icon_path, 'goats')
sheep_map_path = create_animal_map('O', sheep_icon_path, 'sheep')

print(cattle_map_path, goat_map_path, sheep_map_path)


slaughterhouses_map_cattle_de.html slaughterhouses_map_goats_de.html slaughterhouses_map_sheep_de.html


### French version prodcues the html in french language

In [31]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

# Define the animal mappings in French
animal_mapping_french = {
    'A': 'Volaille domestique',
    'B': 'Bovins',
    'C': 'Caprins',
    'L': 'Léporidés (domestiques)',
    'O': 'Moutons',
    'P': 'Porcs',
    'S': 'Équidés',
    'fG': 'Mammifères terrestres en enclos',
    'R': 'Oiseaux coureurs',
    'wA': 'Oiseaux sauvages',
    'wL': 'Léporidés sauvages',
    'wU': 'Ongulés sauvages',
    'wG': 'Mammifères terrestres sauvages'
}

# Correct file paths for the provided icons
cow_icon_path = 'cowhead.png'
goat_icon_path = 'goathead.png'
sheep_icon_path = 'sheephead.png'

# Load the CSV file
file_path = 'slaughterhouse_with_coordinates.csv'
slaughterhouse_data = pd.read_csv(file_path)

# Clean the data: remove rows where 'Tierart' is NaN or not a string, and where Latitude or Longitude are NaN
slaughterhouse_data_cleaned = slaughterhouse_data.dropna(subset=['Tierart', 'Latitude', 'Longitude'])
slaughterhouse_data_cleaned = slaughterhouse_data_cleaned[slaughterhouse_data_cleaned['Tierart'].apply(lambda x: isinstance(x, str))]

# Define the bounding box for Switzerland
switzerland_bounds = [[45.8179, 5.9561], [47.8085, 10.4923]]

# Create a function to generate the map based on the animal type and icon
def create_animal_map(animal_type, icon_path, animal_name):
    # Filter data for the specific animal type
    filtered_data = slaughterhouse_data_cleaned[slaughterhouse_data_cleaned['Tierart'].str.contains(animal_type)]

    # Create a base map centered around Switzerland with max bounds
    map_center = [46.8182, 8.2275]  # Latitude and Longitude of Switzerland
    m = folium.Map(
        location=map_center, 
        zoom_start=7.5,
        min_zoom=7.5, 
        max_bounds=True,
        max_lat=47.8085, min_lat=45.8179,
        max_lon=10.4923, min_lon=5.9561
    )

    # Add max bounds to the map
    m.fit_bounds(switzerland_bounds)
    m.add_child(folium.LatLngPopup())  # Optional: for debugging to see lat/lon

    # Create a marker cluster
    marker_cluster = MarkerCluster().add_to(m)

    # Add markers to the map
    for index, row in filtered_data.iterrows():
        popup_content = (
            f"<b>{row['Firmenname']}</b><br>{row['Adresse']}<br>{int(row['PLZ'])} {row['Ort/Region']}<br>"
            f"Services: {', '.join([animal_mapping_french[type] for type in row['Tierart'].split(', ') if type in animal_mapping_french])}"
        )
        popup = folium.Popup(popup_content, max_width=300)

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup,
            icon=folium.CustomIcon(icon_path, icon_size=(30, 30))
        ).add_to(marker_cluster)

    # Save the map to an HTML file
    output_file_path = f'slaughterhouses_map_{animal_name}_fr.html'
    m.save(output_file_path)

    return output_file_path

# Create the maps for cattle, goats, and sheep
cattle_map_path = create_animal_map('B', cow_icon_path, 'cattle')
goat_map_path = create_animal_map('C', goat_icon_path, 'goats')
sheep_map_path = create_animal_map('O', sheep_icon_path, 'sheep')

print(cattle_map_path, goat_map_path, sheep_map_path)


slaughterhouses_map_cattle_fr.html slaughterhouses_map_goats_fr.html slaughterhouses_map_sheep_fr.html
